<a href="https://www.kaggle.com/code/averma111/pytorch-pss3e18-vanilla?scriptVersionId=135313736" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:

import random
import numpy as np 
import pandas as pd 
import os
import datetime
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F



import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import itertools

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e18/sample_submission.csv
/kaggle/input/playground-series-s3e18/train.csv
/kaggle/input/playground-series-s3e18/test.csv


In [4]:
class Datapreparation(object):
    
    def __init__(self,root_path):
        self.root_path = root_path
        
    def get_dataframe(self,filename):
        return pd.read_csv(os.path.join(self.root_path,filename))
    
    def summary(self,text, df):
        summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
        summary['null'] = df.isnull().sum()
        summary['unique'] = df.nunique()
        summary['min'] = df.min()
        summary['median'] = df.median()
        summary['max'] = df.max()
        summary['mean'] = df.mean()
        summary['std'] = df.std()
        summary['duplicate'] = df.duplicated().sum()
        return summary
    
    
    def random_split_data(self,X,y):
        return train_test_split(X, y,test_size=0.20,random_state=42)

 
    def standardization_data(self,X_data):
        scaler = StandardScaler()
        std_X_data = scaler.fit_transform(X_data)
        return std_X_data
    

    
data = Datapreparation('/kaggle/input/playground-series-s3e18')
train=data.get_dataframe('train.csv')

In [5]:
data.summary('train',train)

,dtypes,null,unique,min,median,max,mean,std,duplicate
id,int64,0,14838,0.000000,7418.500000,14837.000000,7418.500000,4283.505982,0
BertzCT,float64,0,2368,0.000000,290.987941,4069.959780,515.153604,542.456370,0
Chi1,float64,0,1259,0.000000,6.485270,69.551167,9.135189,6.819989,0
Chi1n,float64,0,3157,0.000000,4.052701,50.174588,5.854307,4.647064,0
Chi1v,float64,0,3306,0.000000,4.392859,53.431954,6.738497,5.866444,0
Chi2n,float64,0,3634,0.000000,2.970427,32.195368,4.432570,3.760516,0
Chi2v,float64,0,3725,0.000000,3.242775,34.579313,5.253221,4.925065,0
Chi3v,float64,0,3448,0.000000,1.948613,22.880836,3.418749,3.436208,0
Chi4n,float64,0,2930,0.000000,1.073261,16.072810,1.773472,1.865898,0
EState_VSA1,float64,0,719,0.000000,17.353601,363.705954,29.202823,31.728679,0


In [6]:
y1 = train['EC1']
y2 = train['EC2']
train.drop(columns=['id','EC1','EC2','EC3','EC4','EC5','EC6'],axis=1,inplace=True)
X = train.copy()

In [7]:
X_train,X_val,y1_train,y1_val = data.random_split_data(X,y1)
print('Data splits for EC1 \n',X_train.shape,X_val.shape,y1_train.shape,y1_val.shape)

Data splits for EC1 
 (11870, 31) (2968, 31) (11870,) (2968,)


In [8]:
X_train,X_val,y2_train,y2_val = data.random_split_data(X,y2)
print('Data splits for EC2 \n',X_train.shape,X_val.shape,y2_train.shape,y2_val.shape)

Data splits for EC2 
 (11870, 31) (2968, 31) (11870,) (2968,)


In [9]:
std_X_train = data.standardization_data(X_train)
std_X_val = data.standardization_data(X_val)
print(std_X_train[0],std_X_val[0])

[-0.6460012  -0.49660398 -0.53250602 -0.57261284 -0.5904852  -0.61753263
 -0.59673579 -0.62771573 -0.91411097  0.56344636 -0.56889535 -0.07305243
  0.81469479  1.50215144  0.0217275  -0.57591696 -0.08736038 -0.39329628
  1.34611003 -0.72802128 -0.78622974 -0.51646882  0.1563295   0.31295018
 -0.61487415 -0.5171512  -0.91283104 -0.93428166 -0.94969081 -0.68328343
 -0.68460195] [-0.73157789 -0.89416128 -0.71611727 -0.71782103 -0.74457679 -0.73521665
 -0.8362178  -0.84437652 -0.385045   -0.76779355 -0.74586676  0.06227215
  0.02884275  0.07397899  0.41849986 -0.75437549 -0.10510849 -0.80384358
  0.35858918 -0.61099998 -0.3717161  -0.33573224 -0.47165217 -0.56987761
 -0.62586121 -0.22583258 -0.54067893 -0.28699515 -0.54836477  0.78492852
  0.78337988]


In [10]:
class Tensoroperations():
    
    def __init__(self):
        super(Tensoroperations,self).__init__()
    
    def convert_to_tensor(self,X,y=None):
        X_tensor =  torch.from_numpy(X).float()   
        y_tensor = torch.from_numpy(y).float() 
        return X_tensor,y_tensor
        
    def convert_to_test_tesnor(self,X):
        X_tensor =  torch.from_numpy(X).float()
        return X_tensor
    
    def get_dataloaders(self,train_dataset,val_dataset):
        train_loaders = DataLoader(train_dataset,batch_size=32,shuffle=True)
        val_loaders = DataLoader(val_dataset,batch_size=32)
        return train_loaders,val_loaders
    
    def get_test_dataloaders(self,test_dataset,X_test):
        test_loaders = DataLoader(test_dataset,batch_size=X_test.shape[0])
        return test_loaders
        
        
    
tenops = Tensoroperations()

In [11]:
class CustomDataset(Dataset):
    
    def __init__(self,X_data,y_data=None,is_train=True):
        super().__init__()
        if is_train:
            self.X_data = X_data
            self.y_data = y_data
        else:
            self.X_data=X_train
            
    def __getitem__(self,index):
        return (self.X_data[index],self.y_data[index])
    
    def __len__(self):
        return len(self.X_data)

In [12]:
X_tensor_train,y1_tensor_train = tenops.convert_to_tensor(std_X_train,y1_train.values)
X_tensor_val,y1_tensor_val = tenops.convert_to_tensor(std_X_val,y1_val.values)
print('The training tensor for EC1\n',X_tensor_train,y1_tensor_train)
print('The validation tensor for EC1\n',X_tensor_val,y1_tensor_val)

The training tensor for EC1
 tensor([[-0.6460, -0.4966, -0.5325,  ..., -0.9497, -0.6833, -0.6846],
        [-0.7818, -0.9513, -0.8922,  ..., -1.0127,  0.8178,  0.8161],
        [ 2.1245,  1.5223,  1.7448,  ...,  2.0297, -0.6833, -0.6846],
        ...,
        [-0.2776,  0.0232, -0.3406,  ..., -0.6133, -0.6833, -0.6846],
        [-0.7166, -0.6467, -0.5530,  ..., -0.6577, -0.6833, -0.6846],
        [-0.8225, -0.8068, -0.8795,  ..., -0.8180, -0.6833, -0.6846]]) tensor([1., 1., 1.,  ..., 0., 0., 1.])
The validation tensor for EC1
 tensor([[-0.7316, -0.8942, -0.7161,  ..., -0.5484,  0.7849,  0.7834],
        [-0.2605,  0.2136,  0.2536,  ..., -0.2027, -0.6970, -0.6985],
        [-0.4432, -1.0872, -1.1129,  ..., -0.1914,  2.2669,  2.2653],
        ...,
        [-0.6615, -0.6798, -0.6762,  ..., -0.3160,  0.7849,  0.7834],
        [ 0.7218,  0.9378,  0.8351,  ...,  0.2456, -0.6970, -0.6985],
        [-0.8259, -0.8942, -0.8957,  ..., -0.9963, -0.6970, -0.6985]]) tensor([1., 0., 1.,  ..., 1., 0.,

In [13]:
X_tensor_train,y2_tensor_train = tenops.convert_to_tensor(std_X_train,y2_train.values)
X_tensor_val,y2_tensor_val = tenops.convert_to_tensor(std_X_val,y2_val.values)
print('The training tensor EC2\n',X_tensor_train,y2_tensor_train)
print('The validation tensor Ec2\n',X_tensor_val,y2_tensor_val)

The training tensor EC2
 tensor([[-0.6460, -0.4966, -0.5325,  ..., -0.9497, -0.6833, -0.6846],
        [-0.7818, -0.9513, -0.8922,  ..., -1.0127,  0.8178,  0.8161],
        [ 2.1245,  1.5223,  1.7448,  ...,  2.0297, -0.6833, -0.6846],
        ...,
        [-0.2776,  0.0232, -0.3406,  ..., -0.6133, -0.6833, -0.6846],
        [-0.7166, -0.6467, -0.5530,  ..., -0.6577, -0.6833, -0.6846],
        [-0.8225, -0.8068, -0.8795,  ..., -0.8180, -0.6833, -0.6846]]) tensor([1., 1., 0.,  ..., 1., 1., 1.])
The validation tensor Ec2
 tensor([[-0.7316, -0.8942, -0.7161,  ..., -0.5484,  0.7849,  0.7834],
        [-0.2605,  0.2136,  0.2536,  ..., -0.2027, -0.6970, -0.6985],
        [-0.4432, -1.0872, -1.1129,  ..., -0.1914,  2.2669,  2.2653],
        ...,
        [-0.6615, -0.6798, -0.6762,  ..., -0.3160,  0.7849,  0.7834],
        [ 0.7218,  0.9378,  0.8351,  ...,  0.2456, -0.6970, -0.6985],
        [-0.8259, -0.8942, -0.8957,  ..., -0.9963, -0.6970, -0.6985]]) tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [15]:
n_input_dim = X_train.shape[1]
#Layer size
n_hidden1 = 4  # Number of hidden nodes
n_hidden2 = 2
n_output =  1   # Number of output nodes = for binary classifier

class MultiClassificationNN(torch.nn.Module):
    
    def __init__(self):
        super(MultiClassificationNN, self).__init__()
        self.layer_1 = torch.nn.Linear(n_input_dim, n_hidden1) 
        self.layer_2 = torch.nn.Linear(n_hidden1, n_hidden2)
        self.layer_out = torch.nn.Linear(n_hidden2, n_output) 
        
        
        self.relu = torch.nn.ReLU()
        self.sigmoid =  torch.nn.Sigmoid()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.batchnorm1 = torch.nn.BatchNorm1d(n_hidden1)
        self.batchnorm2 = torch.nn.BatchNorm1d(n_hidden2)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        
        return x
    


model_EC1 = MultiClassificationNN()
model_EC2 = MultiClassificationNN()
print(model_EC1.state_dict())
print(model_EC2.state_dict())

OrderedDict([('layer_1.weight', tensor([[ 8.2011e-02,  3.2315e-02,  1.9249e-02, -7.8593e-02, -3.6137e-03,
         -5.9090e-02, -3.2268e-02,  6.5100e-02, -3.0669e-03,  4.5142e-02,
          1.1615e-01, -4.7788e-02,  1.7272e-01,  5.3252e-02, -3.8916e-02,
         -1.1752e-01, -1.7290e-01, -1.4131e-01, -2.9494e-02,  1.0121e-01,
          4.2926e-02,  1.0384e-01,  1.6157e-01,  9.7131e-02, -5.1818e-02,
         -1.5843e-01,  1.0194e-01,  5.6426e-02,  8.9926e-02,  1.4215e-01,
         -1.0575e-01],
        [ 8.4963e-02,  4.6338e-02, -1.7249e-01,  2.7926e-04, -4.2624e-02,
          5.5870e-02, -8.9300e-03, -1.2896e-01, -6.3465e-02, -1.7108e-01,
         -1.5602e-01,  9.5728e-02,  1.2824e-01, -1.3064e-01,  6.4507e-02,
          1.1303e-01, -1.7458e-01, -8.0593e-02,  1.3945e-01, -9.6799e-02,
         -1.5888e-01, -1.6409e-01, -6.4219e-02, -1.0038e-01,  1.2313e-01,
          1.6725e-01,  9.2550e-02,  9.2971e-02,  1.0378e-01,  6.3568e-02,
          1.2819e-02],
        [ 1.2571e-01, -1.4418e-01,

In [17]:
train1_dataset = CustomDataset(X_tensor_train,y1_tensor_train)
val1_dataset = CustomDataset(X_tensor_val,y1_tensor_val)

train2_dataset = CustomDataset(X_tensor_train,y2_tensor_train)
val2_dataset = CustomDataset(X_tensor_val,y2_tensor_val)

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.BCELoss()
     

In [33]:
dataset = ConcatDataset([train1_dataset, val1_dataset])

num_epochs=10
batch_size=32
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [46]:
class Trainer:
    
    def train_epoch(self,model,device,dataloader,loss_fn,optimizer):
        train_loss,train_correct=0.0,0
        model.train()
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output,labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(output.data, 1)
            train_correct += (predictions == labels).sum().item()
        return train_loss,train_correct


    def valid_epoch(self,model,device,dataloader,loss_fn):
        valid_loss, val_correct = 0.0, 0
        model.eval()
        with torch.no_grad():
          for images, labels in dataloader:
              images,labels = images.to(device),labels.to(device)
              output = model(images)
              loss=loss_fn(output,labels.unsqueeze(1))
              valid_loss+=loss.item()*images.size(0)
              scores, predictions = torch.max(output.data,1)
              val_correct+=(predictions == labels).sum().item()

        return valid_loss,val_correct
    
trainer = Trainer()

In [47]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

In [49]:
class Modeltrain:
    def  call_train(self,model):
        
        for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
            
            print('Fold {}'.format(fold + 1))
            train_sampler = SubsetRandomSampler(train_idx)
            test_sampler = SubsetRandomSampler(val_idx)
            train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
            model.to(device)
            optimizer = optim.Adam(model.parameters(), lr=0.002)
            
            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=trainer.train_epoch(model,device,train_loader,criterion,optimizer)
                test_loss, test_correct=trainer.valid_epoch(model,device,test_loader,criterion)
                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                test_loss = test_loss / len(test_loader.sampler)
                test_acc = test_correct / len(test_loader.sampler) * 100

            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                            test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)  
        
        
mt = Modeltrain()
mt.call_train(model_EC1)

Fold 1


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:10/10 AVG Training Loss:0.578 AVG Test Loss:0.584 AVG Training Acc 33.17 % AVG Test Acc 33.69 %
Fold 2


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:10/10 AVG Training Loss:0.580 AVG Test Loss:0.576 AVG Training Acc 33.32 % AVG Test Acc 32.35 %
Fold 3


...

Epoch:10/10 AVG Training Loss:0.580 AVG Test Loss:0.568 AVG Training Acc 33.39 % AVG Test Acc 31.74 %
Fold 9


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:10/10 AVG Training Loss:0.578 AVG Test Loss:0.579 AVG Training Acc 33.08 % AVG Test Acc 34.52 %
Fold 10


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:10/10 AVG Training Loss:0.578 AVG Test Loss:0.577 AVG Training Acc 33.19 % AVG Test Acc 33.58 %


In [50]:
avg_train_loss = np.mean(history['train_loss'])
avg_test_loss = np.mean(history['test_loss'])
avg_train_acc = np.mean(history['train_acc'])
avg_test_acc = np.mean(history['test_acc'])

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f} \t Average Training Acc: {:.3f} \t Average Test Acc: {:.3f}".format(avg_train_loss,avg_test_loss,avg_train_acc,avg_test_acc))  


Performance of 10 fold cross validation
Average Training Loss: 0.5784 	 Average Test Loss: 0.5786 	 Average Training Acc: 33.186 	 Average Test Acc: 33.581


In [ ]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k=10
for f in range(1,k+1):

     tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
     testl_f.append(np.mean(foldperf['fold{}'.format(f)]['test_loss']))

     ta_f.append(np.mean(foldperf['fold{}'.format(f)]['train_acc']))
     testa_f.append(np.mean(foldperf['fold{}'.format(f)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".f

In [52]:
diz_ep = {'train_loss_ep':[],'test_loss_ep':[],'train_acc_ep':[],'test_acc_ep':[]}

for i in range(num_epochs):
      diz_ep['train_loss_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['train_loss'][i] for f in range(k)]))
      diz_ep['test_loss_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['test_loss'][i] for f in range(k)]))
      diz_ep['train_acc_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['train_acc'][i] for f in range(k)]))
      diz_ep['test_acc_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['test_acc'][i] for f in range(k)]))

KeyError: 'fold1'

In [51]:
# Plot losses
plt.figure(figsize=(10,8))
plt.semilogy(diz_ep['train_loss_ep'], label='Train')
plt.semilogy(diz_ep['val_loss_ep'], label='Vallidation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.grid()
plt.legend()
plt.title('EC1 Loss')
plt.show()
     

NameError: name 'diz_ep' is not defined

<Figure size 1000x800 with 0 Axes>

In [ ]:

# Plot accuracies
plt.figure(figsize=(10,8))
plt.semilogy(diz_ep['train_acc_ep'], label='Train')
plt.semilogy(diz_ep['val_acc_ep'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.grid()
plt.legend()
plt.title('EC1 Accuracy')
plt.show()